In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Based on the provided texts, the purpose of man, as understood through the lens of Vedic philosophy and related traditions, can be summarized as follows:

*   **Attaining Ultimate Salvation/Spiritual Perfection:** The overarching goal is to achieve liberation from the cycle of birth and death and attain a state of eternal bliss. This is often framed as realizing one's connection to the Supreme Being (Vishnu or Krishna).

*   **Self-Realization:** A key aspect of this salvation is understanding one's true self and detaching from the material world. This can be achieved through philosophical inquiry (Sankhya) or devotional practices (Yoga).

*   **Controlling the Senses:** Sense gratification is seen as the root cause of material bondage. Disciplining the senses through practices like yoga, sacrifice, and regulated life (brahmacari, householder life) is crucial for spiritual progress.

*   **Following Dharma:** Adhering to one's prescribed duties and responsibilities within the *varnasrama-dharma* system (community projects and family welfare) is considered essential for enabling individuals to remember Vishnu and achieve salvation.

In essence, the purpose of man, according to these traditions, is to transcend the temporary material existence, realize one's eternal spiritual nature, and achieve union with the divine.


In [ ]:
from src.data_ingestion.pipeline import get_docs
from src.vector_store.load_db import PGVectorDataLoader

docs = get_docs()
pg_vec = PGVectorDataLoader(tablename="gita")
# pg_vec.load_documents(docs)

In [3]:
pg_vec.retrieve("Who is Krishna?")

[Document(id='a78ddcf0-e428-4ac1-a478-7b90bbd8130d', metadata={'Verse': '19', 'Chapter': '9', 'Reference': '9:19', 'Translation': '**O Arjuna, I give heat, and I withhold and send forth the rain. I am immortality, and I am also death personified. Both spirit and matter are in Me.**'}, page_content='**O Arjuna, I give heat, and I withhold and send forth the rain. I am immortality, and I am also death personified. Both spirit and matter are in Me.** is spoken and it can be understood using Krishna, by His different energies, diffuses heat and light through the agency of electricity and the sun. During summer season it is Krishna who checks rain from falling from the sky, and then during the rainy season He gives unceasing torrents of rain. The energy which sustains us by prolonging the duration of our life is Krishna, and Krishna meets us at the end as death. By analyzing all these different energies of Krishna, one can ascertain that for Krishna there is no distinction between matter an

In [4]:
def get_relevant_documents(query: str):
  """
  Retrieves relevant documents from the vectorstore for a given query.

  Args:
    query: The search query.

  Returns:
    A list of relevant documents.
  """
  return vectorstore.similarity_search(query)

In [5]:

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

def get_response_from_query(query: str):
    """
    Retrieves relevant documents and generates a response for a given query
    using newer LangChain patterns.

    Args:
      query: The search query.

    Returns:
      The generated response from the relevant documents.
    """
    # Define a template for the prompt
    template = """Answer the following question based only on the provided context:
    {context}

    Question: {input}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Create a stuff document chain
    # This chain combines documents into a single prompt and passes it to the model
    document_chain = create_stuff_documents_chain(model, prompt)

    # Create a retriever from your vectorstore
    # This retriever will fetch the most relevant documents based on the query
    retriever = vectorstore.as_retriever()

    # Create a retrieval chain
    # This chain first retrieves documents using the retriever,
    # then passes them to the document_chain to generate the response
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    # Invoke the retrieval chain with the query
    # Use .invoke() instead of .run()
    response = retrieval_chain.invoke({"input": query})

    # The response from the retrieval_chain.invoke will be a dictionary.
    # The generated answer is typically in the 'answer' key.
    return response['answer']

In [6]:
get_response_from_query("What is the purpose of life according to the Bhagavad Gita?")

'Based on the provided text, the purpose of life according to the Bhagavad-gita is to be delivered from the nescience of material existence. It also includes realizing the aim of human life, which is to revive our sanatana occupation, or sanatana-dharma, the eternal occupation of the living entity, in relationship with the eternal Supreme Lord.'

In [ ]:
get_response_from_query("What is real religion")

In [ ]:
print(get_response_from_query("How to control my lust?"))

In [ ]:
query = input("Enter your query: ")
print(get_response_from_query(query))